In [31]:
from dash import Dash, dcc, html, clientside_callback, Input, Output, Patch, ctx
import plotly.express as px
import pandas as pd
import numpy as np
import dash_bootstrap_components as dbc
from dash_bootstrap_templates import load_figure_template

In [8]:
df = px.data.gapminder()
df

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
0,Afghanistan,Asia,1952,28.801,8425333,779.445314,AFG,4
1,Afghanistan,Asia,1957,30.332,9240934,820.853030,AFG,4
2,Afghanistan,Asia,1962,31.997,10267083,853.100710,AFG,4
3,Afghanistan,Asia,1967,34.020,11537966,836.197138,AFG,4
4,Afghanistan,Asia,1972,36.088,13079460,739.981106,AFG,4
...,...,...,...,...,...,...,...,...
1699,Zimbabwe,Africa,1987,62.351,9216418,706.157306,ZWE,716
1700,Zimbabwe,Africa,1992,60.377,10704340,693.420786,ZWE,716
1701,Zimbabwe,Africa,1997,46.809,11404948,792.449960,ZWE,716
1702,Zimbabwe,Africa,2002,39.989,11926563,672.038623,ZWE,716


In [26]:
np.sort(df.year.unique())[::-1][0].item()

2007

In [ ]:
import dash
from dash import Dash, dcc, html, clientside_callback, Input, Output, Patch, ctx, ClientsideFunction
import plotly.express as px
import pandas as pd
import numpy as np
import dash_bootstrap_components as dbc
from dash_bootstrap_templates import load_figure_template
import plotly.io as pio


load_figure_template(["minty",  "minty_dark", 'cyborg', 'cyborg_dark'])


color_mode_switch = html.Span(
    [
        dbc.Label(className="fa fa-moon", html_for="color-mode-switch"),
        dbc.Switch( id="color-mode-switch", value=False, className="d-inline-block ms-1", persistence=True),
        dbc.Label(className="fa fa-sun", html_for="color-mode-switch"),
    ]
)


df = px.data.gapminder()

dff = px.data.gapminder().query(f"year == {np.sort(df.year.unique())[::-1][0].item()}")

fig = px.sunburst(
    dff,
    path=["continent", "country"],
    values="pop",
    color="lifeExp",
    hover_data=["iso_alpha"],
    color_continuous_scale="RdBu",
    color_continuous_midpoint=np.average(df["lifeExp"], weights=df["pop"]),
    template='minty'
)
fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))



app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
    html.Div(
        ["Dash Chart Example"],
        id='title_div',
        style={'padding':'10px'},
    ),
    color_mode_switch,

    dbc.Row(
        [
            dbc.Col(
                [
                    html.Label("Select year:"),
                    dcc.Dropdown(
                        id="year",
                        options=df.year.unique(),
                        value=np.sort(df.year.unique())[::-1][0].item(),
                        clearable=False,
                   #     className='dropdown-dark'
                    ),
                ],
                width=3,
            ),
            dbc.Col(
                [dcc.Graph(id="graph", figure=fig)],
                width=9,
            ),
        ]
    )
],    id='root_container',
    className='container')


@app.callback(
    Output("graph", "figure"),

    inputs={
        'input_dict':{
            'dropdown_year': Input("year", "value"),
            'switch_template': Input("color-mode-switch", "value")
        }
    },
    prevent_initial_call=True
)
def generate_graph(input_dict):

    inputs = ctx.args_grouping.input_dict

    if inputs.dropdown_year.triggered:
        dff = px.data.gapminder().query(f"year == {inputs.dropdown_year.value}")
        fig = px.sunburst(
            dff,
            path=["continent", "country"],
            values="pop",
            color="lifeExp",
            hover_data=["iso_alpha"],
            color_continuous_scale="RdBu",
            color_continuous_midpoint=np.average(df["lifeExp"], weights=df["pop"]),
            template='minty' if inputs.switch_template.value else 'minty_dark'
        )
        fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))
        return fig
    elif inputs.switch_template.triggered:
        patched_figure = Patch()
        patched_figure["layout"]["template"] = pio.templates["minty"] if inputs.switch_template.value else pio.templates["minty_dark"]
        return patched_figure

    return dash.no_update


app.clientside_callback(
    """function(switchOn){
           document.documentElement.setAttribute('data-bs-theme', switchOn ? 'light' : 'dark');
           document.querySelector('body').style.backgroundColor = switchOn ? 'white':'rgb(25, 25, 25)' ;

           return window.dash_clientside.no_update
    }""",
    Output("root_container", "id"),
    Input("color-mode-switch", "value"),
)


if __name__ == '__main__':
    app.run(port=8872, debug=True)
